##### This jupyter notebook serves as a tutorial for the code used to run the TWIMLfest 2020 Codenames competition. Please first read through the [README](https://github.com/dhilgart/TWIML_codenames/blob/master/README.md) for an overview of the competition. The README also includes links to the rules of the official board game Codenames (by Vlaada Chvátil)  and a video that shows how to play.

In [1]:
import TWIML_codenames
import numpy as np
import random
from datetime import datetime

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhilg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# How games get played

In order for a game to be created, it needs a few objects provided on instantiation. First, let's 

### Create a TWIML_codenames.Gameboard object:

In [2]:
#In order to create a gameboard, you must provide it with a list of words
wordlist = ['hood','cross','point','bow','pass','plate','cell','theater','bell','scale','straw','press','force','laser','ambulance','thumb','bottle','scorpion','wave','day','revolution','turkey','fair','washer','mass']

In [3]:
#create a gameboard:
gameboard = TWIML_codenames.Gameboard(wordlist)

A gameboard has 3 5x5 numpy arrays that store information about the gameboard. Let's visualize the words on the board:

In [4]:
gameboard.boardwords

array([['bell', 'washer', 'day', 'fair', 'hood'],
       ['thumb', 'revolution', 'scorpion', 'bottle', 'press'],
       ['pass', 'ambulance', 'plate', 'scale', 'force'],
       ['cross', 'turkey', 'straw', 'cell', 'theater'],
       ['laser', 'point', 'bow', 'mass', 'wave']], dtype='<U10')

Now let's visualize the key, which tells the spymasters which words belong to which team. The teams are as follows: 
- 1 = team 1
- 2 = team 2
- 0 = neutral
- -1 = assassin

Note: we do not use blue team and red team. Whichever team goes first is team 1 and has 1 extra word they need to guess.

In [5]:
gameboard.boardkey

array([[-1,  2,  1,  1,  0],
       [ 2,  1,  0,  2,  1],
       [ 1,  0,  1,  2,  1],
       [ 0,  1,  2,  2,  0],
       [ 2,  1,  0,  2,  0]])

Lastly, there is an array that stores a record of which words have been guessed so far in the game. At the beginning of the game, it is all np.NaN:

In [6]:
gameboard.boardmarkers

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

The next thing the TWIML_codenames.Game needs is players! It requires a list of TWIML_codenames.Player objects on team1 and on team2. So, let's

### Create TWIML_codenames.Player objects:

We will need 4 Player objects: 2 for team 1 and 2 for team 2. Let's put them in a list called playerlist. Each Player object will need a player_id. For the competition, your player_ID has been emailed to you. For this demo, let's use player_IDs 1001, 1002, 1003, and 1004.

In [7]:
playerlist=[]
for i in range(4):
    playerlist.append(TWIML_codenames.Player(1001+i))
    print(playerlist[i])

Let's investigate what a TWIML_codenames.Player object contains. It stores:
- the player_id
- the Elo ratings of the player (These will be used to determine the winners. See the README for details)
- the Win-Loss records of the player

In [8]:
playerlist[0].player_id

1001

In [9]:
playerlist[0].Elo

{'Spymaster': 1500.0, 'Operative': 1500.0}

In [10]:
playerlist[0].record

{'Spymaster': {'W': 0, 'L': 0}, 'Operative': {'W': 0, 'L': 0}}

Note that the Elo ratings and records have been pre-populated on instantiation. Both Elo and W-L record are tracked separately for the Spymaster and Operative roles.

We have everything we need to create a game! Let's do it!

### Create a TWIML_codenames.Game object:

In [11]:
# first split the players into teams:
team1 = playerlist[:2]
team2 = playerlist[2:]

In [12]:
# then create the game:
my_game = TWIML_codenames.Game(gameboard = gameboard, team1 = team1, team2 = team2)

The game is ready! 

When a game is instantiated, it sets all of its internal variables to prepare for the Team 1 Spymaster to give the first clue. Let's take a look:

In [13]:
my_game.curr_team

1

In [14]:
my_game.waiting_on

'spymaster'

Let's not keep the game waiting too long! We need to

### Feed the game a clue:

In order to do so, let's load my_model.py

In [15]:
import my_model as model

Now all we need to do to figure out what clue to feed the game is call model.generate_clue:

In [16]:
# when interacting with a game on the server, client_run.py first calls game.solicit_clue_inputs(): 
pull_team_num, pull_gameboard = my_game.solicit_clue_inputs()
# you can skip this step when running locally by just using my_game.curr_team and my_game.gameboard as in the next cell:

In [17]:
%%time
# this can take awhile with the template bot. On my laptop it takes about 3.5 minutes. 
# You may want to reduce the size of the sample taken from nounlist to improve the runtime.
# Reduce the 1000 in this line in my_model.py: np.random.choice(full_candidates,1000, replace=False)
clue_word, clue_count = model.generate_clue(game_id=1, # note: game_id is a required input of model.generate_clue so that you can keep track of which game is asking for a clue. Each game on the server will have a unique, 6-digit integer for the game_id. For the purposes of this demo, we will just give it a game_id of 1.
                                            team_num=my_game.curr_team, 
                                            gameboard=my_game.gameboard)

C:\Users\dhilg\OneDrive\Documents\Dan\Python\Codenames\git\TWIML_codenames\my_model.py:54: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  return 1 - tokens[0].similarity(tokens[1])


Wall time: 3min 56s


Let's take a look at what clue_word and clue_count it generated:

In [18]:
print(f'My bot says: "{clue_word} for {clue_count}!"')

My bot says: "rush for 1!"


Can you tell what word(s) it wants the operative to guess with that clue? Let's look at the board:

In [19]:
print(my_game.gameboard.boardwords)
print(my_game.gameboard.boardkey)

[['bell' 'washer' 'day' 'fair' 'hood']
 ['thumb' 'revolution' 'scorpion' 'bottle' 'press']
 ['pass' 'ambulance' 'plate' 'scale' 'force']
 ['cross' 'turkey' 'straw' 'cell' 'theater']
 ['laser' 'point' 'bow' 'mass' 'wave']]
[[-1  2  1  1  0]
 [ 2  1  0  2  1]
 [ 1  0  1  2  1]
 [ 0  1  2  2  0]
 [ 2  1  0  2  0]]


Ok, let's feed the clue back to the game:

In [20]:
my_game.clue_given(clue_word, clue_count)

Let's check the status:

In [21]:
my_game.curr_team

1

In [22]:
my_game.waiting_on

'operative'

The game is ready for Team 1's Operative (i.e. guesser) to respond. 
As above, we could pull the inputs the guesser will need using my_game.solicit_guesses_inputs(), but we will skip that for this demo. Time to

### Feed the game a list of guesses:

First, ask the model generate the list:

In [23]:
guesses = model.generate_guesses(game_id = 1, # see comment on game_id when we generated the clues 
                                 team_num = my_game.curr_team, 
                                 clue_word = my_game.curr_clue_word, 
                                 clue_count = my_game.curr_clue_count, 
                                 unguessed_words = my_game.gameboard.unguessed_words(), # gameboard.unguessed_words() returns a 1-D list of all the words on the board that have not yet been guessed
                                 boardwords = my_game.gameboard.boardwords, 
                                 boardmarkers = my_game.gameboard.boardmarkers)
# note that the gameboard is not passed to the guesser function in whole. If it were, the guesser would have access to the key!

Let's take a look at what the bot generated. Does it match with what you thought above?

In [24]:
guesses

['pass']

Send it back to the game:

In [25]:
my_game.guesses_given(guesses)

Let's see what happened. Check the boardmarkers and the boardwords. Did the bot get it right?

In [26]:
print(my_game.gameboard.boardmarkers)
print(my_game.gameboard.boardwords)

[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [ 1. nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]
[['bell' 'washer' 'day' 'fair' 'hood']
 ['thumb' 'revolution' 'scorpion' 'bottle' 'press']
 ['pass' 'ambulance' 'plate' 'scale' 'force']
 ['cross' 'turkey' 'straw' 'cell' 'theater']
 ['laser' 'point' 'bow' 'mass' 'wave']]


Let's check the status again:

In [27]:
my_game.curr_team

2

In [28]:
my_game.waiting_on

'spymaster'

Team 1's turn is done! The game is ready for Team 2's Spymaster to give a clue!

We don't need to go through each guess one at a time. Let's automate the rest of the game:

### Simulate the rest of the game:

In [29]:
start_time=datetime.now()
while my_game.game_completed == False:
    # here we ignore that team 1 and team 2 will have different models and assume that regardless 
    # of which team's turn it is, we use the same model for the spymaster and the operative
    
    if my_game.waiting_on == 'spymaster':
        clue_start_time=datetime.now()
        clue_word, clue_count = model.generate_clue(game_id=1,
                                                    team_num=my_game.curr_team, 
                                                    gameboard=my_game.gameboard)
        print(f'Clue generated. Duration = {datetime.now()-clue_start_time}. Total elapsed time = {datetime.now()-start_time}')
        print(f'My bot says: "{clue_word} for {clue_count}!"')
        my_game.clue_given(clue_word, clue_count)
    else:
        guess_start_time=datetime.now()
        guesses = model.generate_guesses(game_id = 1,
                                         team_num = my_game.curr_team, 
                                         clue_word = my_game.curr_clue_word, 
                                         clue_count = my_game.curr_clue_count, 
                                         unguessed_words = my_game.gameboard.unguessed_words(),
                                         boardwords = my_game.gameboard.boardwords, 
                                         boardmarkers = my_game.gameboard.boardmarkers)
        print(f'Guesses generated. Duration = {datetime.now()-guess_start_time}. Total elapsed time = {datetime.now()-start_time}')
        print(f'guesses = {guesses}')
        my_game.guesses_given(guesses)

Clue generated. Duration = 0:03:32.983610. Total elapsed time = 0:03:32.984611
My bot says: "finger for 1!"
Guesses generated. Duration = 0:00:00.206139. Total elapsed time = 0:03:33.191751
guesses = ['thumb']
Clue generated. Duration = 0:03:21.912924. Total elapsed time = 0:06:55.105672
My bot says: "morning for 1!"
Guesses generated. Duration = 0:00:00.177001. Total elapsed time = 0:06:55.282673
guesses = ['day']
Clue generated. Duration = 0:03:26.804228. Total elapsed time = 0:10:22.086901
My bot says: "phone for 1!"
Guesses generated. Duration = 0:00:00.311182. Total elapsed time = 0:10:22.401086
guesses = ['cell']
Clue generated. Duration = 0:03:23.674126. Total elapsed time = 0:13:46.076203
My bot says: "thought for 1!"
Guesses generated. Duration = 0:00:00.142996. Total elapsed time = 0:13:46.219199
guesses = ['point']
Clue generated. Duration = 0:02:29.595017. Total elapsed time = 0:16:15.814216
My bot says: "wine for 1!"
Guesses generated. Duration = 0:00:00.134035. Total elap

There you have it! A completed game! But that's not all. There's something else we haven't talked about that you might be interested in: the game log!

# The game log

A TWIML_codenames.Game object is always created with a logger object. If no logger is supplied at initialization, a TWIML_codenames.LocalLogger object will be created for the game. LocalLogger stores the gamelog in memory. For games played on the server, a TWIML_codenames_API_server.MongoLogger is supplied at Game initialization, so the log is stored to a MongoDB. The server game logs can be retrieved using the @app.get("http<span>://twiml-codenames.herokuapp</span>.com/{game_id}/log/") API endpoint. Either way, the logger stores the same information as a multi-nested dict.

Let's take a look at the log for this game:

In [30]:
my_game.logger.game_log # this command only works for LocalLogger, not for MongoLogger

{'in_progress': False,
 'events': [{'event': 'clue_given',
   'timestamp': datetime.datetime(2020, 10, 16, 13, 37, 52, 360643),
   'team_num': 1,
   'clue_word': 'rush',
   'clue_count': 1,
   'legal_clue': 'Yes'},
  {'event': 'guess made',
   'timestamp': datetime.datetime(2020, 10, 16, 13, 37, 52, 649161),
   'team_num': 1,
   'word_guessed': 'pass',
   'result': 1},
  {'event': 'end guessing',
   'timestamp': datetime.datetime(2020, 10, 16, 13, 37, 52, 650163),
   'reason': 'no more guesses provided'},
  {'event': 'clue_given',
   'timestamp': datetime.datetime(2020, 10, 16, 13, 41, 25, 698781),
   'team_num': 2,
   'clue_word': 'finger',
   'clue_count': 1,
   'legal_clue': 'Yes'},
  {'event': 'guess made',
   'timestamp': datetime.datetime(2020, 10, 16, 13, 41, 25, 905917),
   'team_num': 2,
   'word_guessed': 'thumb',
   'result': 2},
  {'event': 'end guessing',
   'timestamp': datetime.datetime(2020, 10, 16, 13, 41, 25, 905917),
   'reason': 'no more guesses provided'},
  {'even

Lots of information there! Let's review.
These are the keys in the top level of the log dict:
- in_progress : True or False
- events : this is where all of the events of the game are recorded (more detail below)
- boardwords : the array from the gameboard
- boardkey : the array from the gameboard
- teams : the player_id for each player on each team
- winning team : the information about each winner's Elo change after the game
- losing team : the information about each loser's Elo change after the game
- start time : in utc
- end time : in utc
- boardmarkers : the final status of the board revealing which words were guessed

Note: not all of this information is present until the game is completed. However, you can still pull a log in the middle of a game. You can even pull a log from the MongoDB in the middle of the game using the API endpoint mentioned above. Also, note that when pulling a log from the server, some of this information is scrubbed from the log: information that you would not have had access to based on your role.

So what's in 'events'?

Events is a list of dicts. Each event is a dict. New events are appended to the end of the list, so the list is in chronological order. Here is what gets recorded as an event:
- clue_given - records the word, the count, and whether the clue was a legal clue. If it wasn't, includes an explanation why not.
- guess made - records the word guessed and the team that word belonged to. Note this is not the same as the guesses list. Guesses are made one at a time and each is recorded individually
- guess skipped: guess not in unguessed_words - records the word that was attempted but did not exist
- end guessing - can be for a number of reasons which are all recorded: illegal clue given, zero guesses provided, incorrect guess made, no more guesses provided, or num guesses provided exceeded clue_count+1
- game over - records the reason why the game ended

Use this information as you wish!